In [ ]:
import datetime
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from openbustools.traveltime import grid, data_loader
from openbustools import plotting, spatial, standardfeeds

### Metadata

In [ ]:
#TODO: Add summary statistic estimations from the full datasets

In [ ]:
network_name = 'kcm'
data_folders = ['../data/kcm_realtime/processed/', '../data/atb_realtime/processed/']
train_date = '2023_03_15'
train_n = 3
train_dates = standardfeeds.get_date_list(train_date, int(train_n))
train_dataset = data_loader.DictDataset(data_folders, train_dates)
data_df = train_dataset.data
data_df.head()

### Traces

In [ ]:
# Sample one trajectory for plotting, use KCM network
sample = data_df[data_df['data_folder']=='../data/kcm_realtime/processed/'].sample(1)
sample_trip_id = sample['trip_id'].iloc[0]
sample_file = sample['file'].iloc[0]
sample_df = data_df[(data_df['trip_id']==sample_trip_id)&(data_df['file']==sample_file)]
sample_df = sample_df[sample_df['file']==sample_file]
sample_df = sample_df.to_crs('EPSG:32148').reset_index()
sample_df.head()

In [ ]:
plotting.formatted_shingle_scatterplot(sample_df)

### Feature Distributions

In [ ]:
plotting.formatted_feature_distributions_histplot(data_df)

### Grid

In [ ]:
g = train_dataset.grids['../data/kcm_realtime/processed/']['2023_03_15']

In [ ]:
res = grid.convert_to_frames(g)

In [ ]:
plotting.formatted_grid_animation(res[500:600,:,:])